In [2]:
import os
import glob


In [44]:
path_img = "/home/user/Documents/dataset/Mindboggle101/mindboggle/image_in_MNI152_normalized"
path_seg = "/home/user/Documents/dataset/Mindboggle101/mindboggle/label_31_reID_merged"
img_file = []
seg_file = [] 
files = glob.glob(os.path.join(path_img,"*.nii.gz"))
for file in files:
    if "flipped" not in file:
        img_file.append(file)
        tmp = os.path.split(file)
        tmp = os.path.join(path_seg,tmp[1])
        seg_file.append(tmp)


In [50]:
list(range(1,61))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]